# EQUITY VANILLA EUROPEAN-STYLE OPTIONS

Valuation and Risk of a simple Vanilla Equity Option

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.equity import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  26 Nov 2020 at 02:02 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In the library products are created and then valued. We should be able to vectorise the product inputs and the valuation inputs separately and jointly.

# 1 - Define and value the Call Options using Vectorised Product Inputs

We consider an equity call option. The inputs are:
1) Expiry date
2) Strike price
3) Option Type (call or put)

We start with the strikes as this is the easiest.

I just define some initial non-product inputs

In [3]:
stockPrice = 100.0
volatility = 0.20
interestRate = 0.05
dividendYield = 0.0

In [4]:
valueDate = FinDate(20,1, 2020)

In [5]:
discountCurve = FinDiscountCurveFlat(valueDate, interestRate, FinFrequencyTypes.CONTINUOUS)

In [6]:
model = FinModelBlackScholes(volatility)

## Vectorising the Strikes

In [7]:
expiryDate = FinDate(20, 6, 2020)

In [8]:
strikePrice = 100.0

In [9]:
callOption = FinEquityVanillaOption(expiryDate, strikePrice, FinOptionTypes.EUROPEAN_CALL)

In [10]:
callOption

OBJECT TYPE: FinEquityVanillaOption
EXPIRY DATE: 20-JUN-2020
STRIKE PRICE: 100.0
OPTION TYPE: FinOptionTypes.EUROPEAN_CALL
NUMBER: 1.0

Let's do a valuation

In [11]:
callOption.value(valueDate, stockPrice, discountCurve, dividendYield, model)

6.185689425036813

Now we vectorise the strikes

In [12]:
strikePrice = np.linspace(100,150,10)

In [13]:
callOption = FinEquityVanillaOption(expiryDate, strikePrice, FinOptionTypes.EUROPEAN_CALL)

In [14]:
callOption

OBJECT TYPE: FinEquityVanillaOption
EXPIRY DATE: 20-JUN-2020
STRIKE PRICE: [100.         105.55555556 111.11111111 116.66666667 122.22222222
 127.77777778 133.33333333 138.88888889 144.44444444 150.        ]
OPTION TYPE: FinOptionTypes.EUROPEAN_CALL
NUMBER: 1.0

Now see if this can be vectorised

In [15]:
callOption.value(valueDate, stockPrice, discountCurve, dividendYield, model)

array([6.18568943, 3.71227685, 2.07068289, 1.07658238, 0.52399285,
       0.23997686, 0.1039734 , 0.04284594, 0.01687943, 0.00638777])

Yep this works.

## Vectorising the Expiry Dates FAILS

In [16]:
expiryDates = valueDate.addYears(np.linspace(0.5,10,20))

In [17]:
expiryDates

[20-JUL-2020,
 20-JAN-2021,
 20-JUL-2021,
 20-JAN-2022,
 20-JUL-2022,
 20-JAN-2023,
 20-JUL-2023,
 20-JAN-2024,
 20-JUL-2024,
 20-JAN-2025,
 20-JUL-2025,
 20-JAN-2026,
 20-JUL-2026,
 20-JAN-2027,
 20-JUL-2027,
 20-JAN-2028,
 20-JUL-2028,
 20-JAN-2029,
 20-JUL-2029,
 20-JAN-2030]

Make the strike a single number

In [18]:
strikePrice = 100.0

In [19]:
callOption = FinEquityVanillaOption(expiryDates, strikePrice, FinOptionTypes.EUROPEAN_CALL)

In [20]:
callOption

OBJECT TYPE: FinEquityVanillaOption
EXPIRY DATE: [20-JUL-2020, 20-JAN-2021, 20-JUL-2021, 20-JAN-2022, 20-JUL-2022, 20-JAN-2023, 20-JUL-2023, 20-JAN-2024, 20-JUL-2024, 20-JAN-2025, 20-JUL-2025, 20-JAN-2026, 20-JUL-2026, 20-JAN-2027, 20-JUL-2027, 20-JAN-2028, 20-JUL-2028, 20-JAN-2029, 20-JUL-2029, 20-JAN-2030]
STRIKE PRICE: 100.0
OPTION TYPE: FinOptionTypes.EUROPEAN_CALL
NUMBER: 1.0

In [21]:
callOption.value(valueDate, stockPrice, discountCurve, dividendYield, model)

TypeError: unsupported operand type(s) for -: 'list' and 'FinDate'

This fails as we do not YET have the ability to vectorise the subtraction of dates.

## Vectorising the Enum Type

In [22]:
optionTypes = list(FinOptionTypes)[0:2]

In [23]:
optionTypes

[<FinOptionTypes.EUROPEAN_CALL: 1>, <FinOptionTypes.EUROPEAN_PUT: 2>]

In [25]:
FinEquityVanillaOption(expiryDate, strikePrice, optionTypes).value(valueDate, stockPrice, discountCurve, dividendYield, model)

FinOptionTypes.EUROPEAN_CALL
FinOptionTypes.EUROPEAN_PUT


array([6.18568943, 4.13059756])

This fails due to validation checks.

Copyright (c) 2020 Dominic O'Kane